In [23]:
import pandas as pd
import numpy as np
import os
import warnings
from os import listdir

In [22]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,28 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_Р”РёРјР°_Рё_РќР°СЃС‚СЏ_5c8o19
H2O cluster total nodes:,1
H2O cluster free memory:,3.533 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [2]:
X_test = pd.read_csv('X_test.csv')

In [3]:
X_test['y'] = -1


In [26]:
df = X_test.head(5)

In [ ]:
error_id = []

In [55]:
def predict(x):
    path_to_model = str(int(x.meter)) + '/' + str(int(x.building_id))
    
    if not os.path.exists(path_to_model):
        print("ERROR_ID NOT EXIST: " + str(int(x.building_id)))
        error_id.append(x.building_id)
        return x
    
    files = listdir(path_to_model)
    
    if len(files) != 1:
        print("ERROR_ID"+ str(int(x.building_id)) +"FILES COUNT: " + str(len(files)))
        error_id.append(x.building_id)
        return x
    
    full_path = path_to_model + '/' + files[0]
    
    model =  h2o.load_model(full_path)
    
    x_prep = x.drop(['meter', 'building_id', 'y', 'row_id'])
#     df_prep = pd.DataFrame(np.array([list(x_prep)]), columns = list(x_prep.index))
    h2o_x_prep = h2o.H2OFrame(np.array([list(x_prep)]), column_names = list(x_prep.index))
    y = model.predict(h2o_x_prep)
    
    del model, x_prep, path_to_model, full_path, h2o_x_prep
    
    print(y)
    return x

In [56]:
df.apply(predict, axis=1)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
5.16189


H2OResponseError: (Server error java.lang.IllegalArgumentException:
  Error: Cannot find Builder for algo url name xgboost
  Request: POST /99/Models.bin/
    data: {'dir': '0/1/XGBoost_grid_1_AutoML_20191203_194843_model_1'}
, 'occurred at index 1')